ENS 491-492- GRADUATION PROJECT TASK 1

Will my question get answered? 

Classification task will be applied on stackexchange AI data

Importing libraries

In [1]:
import gensim
import pandas as pd
import warnings
from bs4 import BeautifulSoup
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
!pip install category_encoders
from sklearn.metrics import mean_squared_error 
from sklearn import preprocessing
import xml.etree.ElementTree as et
import re
import warnings
warnings.filterwarnings("ignore")
from itertools import product
from os.path import join
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing


İmporting nltk library for text data cleaning process

In [2]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 


import matplotlib.pyplot as plt
import io
import unicodedata
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Utku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Utku\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Utku\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


 
Data Gathering and Implementation of Data Frame



In [3]:
xtree = et.parse('Posts.xml')

In [4]:
xroot = xtree.getroot()

In [5]:
dfCols = ["Closed Date", "Favorite Count", "Comment Count", "Answer Count", "Tags", "Title",
          "Last Activity Date", "Owner User ID", "Body", "View Count", "Score", "Creation Date", "Post Type ID", 
          "ID", "Parent ID", "Last Edit Date", "Last Editor User ID", "Accepted Answer ID"]
dfRows = []

In [6]:
for node in xroot:
    closedDate = node.attrib.get("ClosedDate")
    favCount = node.attrib.get("FavoriteCount")
    commentCount = node.attrib.get("CommentCount")
    ansCount = node.attrib.get("AnswerCount")
    tags = node.attrib.get("Tags")
    title = node.attrib.get("Title")
    lastActDate = node.attrib.get("LastActivityDate")
    ownerUserID = node.attrib.get("OwnerUserId")
    body = node.attrib.get("Body")
    viewCount = node.attrib.get("ViewCount") 
    score = node.attrib.get("Score") 
    creationDate = node.attrib.get("CreationDate") 
    postTypeID = node.attrib.get("PostTypeId") 
    ID = node.attrib.get("Id") 
    parentID = node.attrib.get("ParentId") 
    lastEditDate = node.attrib.get("LastEditDate") 
    lastEditorUserID = node.attrib.get("LastEditorUserId") 
    acceptedAnswerID = node.attrib.get("AcceptedAnswerID")
    
    dfRows.append({"Closed Date": closedDate, "Favorite Count": favCount, "Comment Count": commentCount,
                     "Answer Count": ansCount, "Tags": tags, "Title": title, "Last Activity Date": lastActDate,
                     "Owner User ID": ownerUserID, "Body": body, "View Count": viewCount, "Score": score, 
                    "Creation Date": creationDate, "Post Type ID": postTypeID, "ID": ID, "Parent ID": parentID,
                    "Last Edit Date": lastEditDate, "Last Editor User ID": lastEditorUserID, "Accepted Answer ID": acceptedAnswerID})

In [7]:
out = pd.DataFrame(dfRows, columns=dfCols)


In [8]:
out = out.fillna(0)

Changing the data types of the parameters

In [9]:
out['Creation Date'] = pd.to_datetime(out['Creation Date'])
out['Last Activity Date'] = pd.to_datetime(out['Last Activity Date'])
out['Last Edit Date'] = pd.to_datetime(out['Last Edit Date'])
out['Comment Count'] = out['Comment Count'].astype(int)
out['Owner User ID'] = out['Owner User ID'].astype(int)
out['Post Type ID'] = out['Post Type ID'].astype(int)
out['Score'] = out['Score'].astype(int)
out['Favorite Count'] = out['Favorite Count'].astype(int)
out['Answer Count'] = out['Answer Count'].astype(int)
out['View Count'] = out['View Count'].astype(int)


In [10]:
out.dtypes


Closed Date                    object
Favorite Count                  int32
Comment Count                   int32
Answer Count                    int32
Tags                           object
Title                          object
Last Activity Date     datetime64[ns]
Owner User ID                   int32
Body                           object
View Count                      int32
Score                           int32
Creation Date          datetime64[ns]
Post Type ID                    int32
ID                             object
Parent ID                      object
Last Edit Date         datetime64[ns]
Last Editor User ID            object
Accepted Answer ID              int64
dtype: object

Categorizing Time Data into Morning/Midday/Evening/Night

**According to UTC**

05:00-10:00 Morning   -->1     24:00 - 05:00 NY Night

11:00 16:00 Midday    -->2     06:00 - 11:00 NY Morning

17:00 22:00 Afternoon -->3     12:00 - 17:00 NY Midday

23:00 04:00 Night     -->4     18:00 - 23:00 NY Evening

In [11]:
def get_part_of_day(x):
    return (
        "1" if 5 <= int(x.strftime('%H')) <= 10
        else
        "2" if 11<=  int(x.strftime('%H'))<= 16
        else
        "3" if 17 <= int(x.strftime('%H'))  <= 22
        else
        "4"
    )
    
out["Creation Date"] = out["Creation Date"].apply(get_part_of_day)
out.head(5)

,Closed Date,Favorite Count,Comment Count,Answer Count,Tags,Title,Last Activity Date,Owner User ID,Body,View Count,Score,Creation Date,Post Type ID,ID,Parent ID,Last Edit Date,Last Editor User ID,Accepted Answer ID
0,0,1,0,3,<neural-networks><backpropagation><terminology...,"What is ""backprop""?",2019-11-16 17:56:22.093,8,"<p>What does ""backprop"" mean? Is the ""backprop...",436,8,2,1,1,0,2019-11-16 17:56:22.093,2444,0
1,0,1,0,3,<neural-networks><machine-learning><statistica...,How does noise affect generalization?,2019-02-23 22:36:37.133,8,<p>Does increasing the noise in data help to i...,546,10,2,1,2,0,2019-02-23 22:36:19.090,2444,0
2,0,0,0,0,0,0,2016-08-02 15:40:24.820,4,"<p>""Backprop"" is the same as ""backpropagation""...",0,13,2,2,3,1,1970-01-01 00:00:00.000,0,0
3,0,10,0,4,<deep-network><search><neurons>,How to find the optimal number of neurons per ...,2018-10-18 10:45:15.213,8,"<p>When you're writing your algorithm, how do ...",930,29,2,1,4,0,2018-10-18 10:45:15.213,10135,0
4,2016-08-02T16:27:32.070,0,4,2,<python><mindstorms>,How to program AI in Mindstorms,2018-10-18 10:44:51.623,5,<p>I have a LEGO Mindstorms EV3 and I'm wonder...,2692,-3,2,1,5,0,2018-10-18 10:44:51.623,10135,0


One Hot Encoding implemented on the date data that had been encoded with label encoding


In [12]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories = "auto",sparse = False, drop = "first")
# One-Hot-Encode Class column of df
temp = ohe.fit_transform(out[["Creation Date"]])


# Converting into dataframe
ohe_column = pd.DataFrame(temp, columns = ["CreationDate_1","CreationDate_2","CreationDate_3"])
#containating ohe column
out= pd.concat([out,ohe_column],axis = 1)


Demonstration of dataframe after one hot encoding applied

In [13]:
out.tail(3)

,Closed Date,Favorite Count,Comment Count,Answer Count,Tags,Title,Last Activity Date,Owner User ID,Body,View Count,...,Creation Date,Post Type ID,ID,Parent ID,Last Edit Date,Last Editor User ID,Accepted Answer ID,CreationDate_1,CreationDate_2,CreationDate_3
13816,0,0,5,0,0,0,2020-02-29 21:18:06.833,2193,"<ol>\n<li>It's not possible, as in the chain i...",0,...,3,2,18333,18332,2020-02-29 21:18:06.833,2193,0,0.0,1.0,0.0
13817,0,0,0,0,<convolutional-neural-networks><object-detection>,How to adapt MTCNN to large images with relati...,2020-02-29 19:20:08.860,16871,<p>This question could be generalised to how t...,8,...,3,1,18334,0,1970-01-01 00:00:00.000,0,0,0.0,1.0,0.0
13818,0,0,0,0,<machine-learning><math><svm>,What are the variables used in a Gaussian radi...,2020-03-01 02:00:34.490,29877,<p>If I have the Gaussian kernel</p>\n\n<p><sp...,4,...,4,1,18336,0,2020-03-01 02:00:34.490,2444,0,0.0,0.0,1.0


Those who has Post Type Id= 1 are questions in this task only questions data will be used

In [14]:
out = out[(out['Post Type ID'] == 1)]

In [15]:
out=out.drop(axis=1,columns=["Post Type ID"])

Since our question is to classify whether the question will get answered , we have labeled data according to it is answered or not by checking answer counts of the questions 

In [16]:
out['IsAnswered'] = np.where(out['Answer Count']>0, '1', '0')

Tag count is calculated and used as a parameter in dataframe 


In [17]:
out['Tag Count'] = out['Tags'].str.count('<')

q (questions abbreviation) data frame consist of the parameters that will be used from Posts.xml data


In [18]:
q = out[["CreationDate_1","CreationDate_2","CreationDate_3",'Owner User ID','Title','Body','ID','Tags','Tag Count','IsAnswered']]


10 parameters will be used from Posts.xml data and ID (question id) will be removed later .It was kept in this stage to identify questions .

In [19]:
print(q.shape)

(5805, 10)


In [20]:
q

,CreationDate_1,CreationDate_2,CreationDate_3,Owner User ID,Title,Body,ID,Tags,Tag Count,IsAnswered
0,1.0,0.0,0.0,8,"What is ""backprop""?","<p>What does ""backprop"" mean? Is the ""backprop...",1,<neural-networks><backpropagation><terminology...,4,1
1,1.0,0.0,0.0,8,How does noise affect generalization?,<p>Does increasing the noise in data help to i...,2,<neural-networks><machine-learning><statistica...,4,1
3,1.0,0.0,0.0,8,How to find the optimal number of neurons per ...,"<p>When you're writing your algorithm, how do ...",4,<deep-network><search><neurons>,3,1
4,1.0,0.0,0.0,5,How to program AI in Mindstorms,<p>I have a LEGO Mindstorms EV3 and I'm wonder...,5,<python><mindstorms>,2,1
5,1.0,0.0,0.0,29,Are humans intelligent according to the defini...,<p>Given the following definition of an intell...,6,<philosophy><definitions><intelligent-agent>,3,1
6,1.0,0.0,0.0,26,"Why does Stephen Hawking say ""Artificial Intel...",<p>This quote by Stephen Hawking has been in h...,7,<intelligent-agent>,1,1
9,1.0,0.0,0.0,8,What is fuzzy logic?,<p>I'm new to A.I. and I'd like to know in sim...,10,<deep-network><terminology><fuzzy-logic>,3,1
12,1.0,0.0,0.0,38,Can a single neural network handle recognizing...,"<p>In particular, an embedded computer (with l...",13,<neural-networks><image-recognition>,2,1
14,1.0,0.0,0.0,9,"Is the Turing Test, or any of its variants, a ...","<p>The <a href=""https://en.wikipedia.org/wiki/...",15,<turing-test><strong-ai><intelligent-agent><we...,4,1
15,1.0,0.0,0.0,8,"What is ""early stopping"" in machine learning?","<p>What is <a href=""https://en.wikipedia.org/w...",16,<deep-learning><definitions><overfitting><regu...,5,1


xml to dict library was used to convert data to data frame, since some failure accured using xml e-tree library 

In [21]:
!pip install xmltodict
import xmltodict
path = 'Users.xml'
xmlDict = xmltodict.parse(et.tostring(et.parse(path).getroot()))

In [22]:
a = (list(xmlDict.items())[0])[1]
b= (list(a.items())[0])[1]

This is the initial version of the dataframe

In [23]:
users= pd.DataFrame.from_dict(b)
users.columns = users.columns.str[1:]
users.head(5)

,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,Id,LastAccessDate,Location,Reputation,UpVotes,Views,ProfileImageUrl,WebsiteUrl
0,"<p>Hi, I'm not really a person.</p>\n<p>I'm a ...",-1,2016-08-02T00:14:10.580,Community,876,-1,2016-08-02T00:14:10.580,on the server farm,1,4,0,NaN,NaN
1,\n\n<p>Developer at Stack Overflow focusing on...,37099,2016-08-02T15:36:45.333,Adam Lear,0,1,2019-11-05T22:06:40.680,"New York, NY",101,0,151,https://i.stack.imgur.com/SMEGn.jpg?s=128&g=1,NaN
2,<p>I am the Architecture Lead for Stack Overfl...,7598,2016-08-02T15:36:48.397,Nick Craver,0,2,2016-11-30T22:09:22.300,"Winston-Salem, NC",101,0,11,https://i.stack.imgur.com/nGCYr.jpg?s=128&g=1,https://nickcraver.com/blog/
3,<p>Developer on the Stack Overflow team. Find...,2,2016-08-02T15:38:01.633,Geoff Dalgas,0,3,2018-01-09T03:22:26.573,"Corvallis, OR, United States",101,0,12,https://i.stack.imgur.com/nDllk.png,http://stackoverflow.com
4,"<p><a href=""https://stackexchange.com/users/16...",169656,2016-08-02T15:38:21.100,Franck Dernoncourt,2,4,2020-02-24T23:40:36.403,"Adobe Research, San Jose, CA, USA",1736,15,110,https://i.stack.imgur.com/Z99mk.jpg?s=128&g=1,http://www.francky.me


Nan Values are filled with "0" before any preprocessing

In [24]:
users.fillna(0)

,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,Id,LastAccessDate,Location,Reputation,UpVotes,Views,ProfileImageUrl,WebsiteUrl
0,"<p>Hi, I'm not really a person.</p>\n<p>I'm a ...",-1,2016-08-02T00:14:10.580,Community,876,-1,2016-08-02T00:14:10.580,on the server farm,1,4,0,0,0
1,\n\n<p>Developer at Stack Overflow focusing on...,37099,2016-08-02T15:36:45.333,Adam Lear,0,1,2019-11-05T22:06:40.680,"New York, NY",101,0,151,https://i.stack.imgur.com/SMEGn.jpg?s=128&g=1,0
2,<p>I am the Architecture Lead for Stack Overfl...,7598,2016-08-02T15:36:48.397,Nick Craver,0,2,2016-11-30T22:09:22.300,"Winston-Salem, NC",101,0,11,https://i.stack.imgur.com/nGCYr.jpg?s=128&g=1,https://nickcraver.com/blog/
3,<p>Developer on the Stack Overflow team. Find...,2,2016-08-02T15:38:01.633,Geoff Dalgas,0,3,2018-01-09T03:22:26.573,"Corvallis, OR, United States",101,0,12,https://i.stack.imgur.com/nDllk.png,http://stackoverflow.com
4,"<p><a href=""https://stackexchange.com/users/16...",169656,2016-08-02T15:38:21.100,Franck Dernoncourt,2,4,2020-02-24T23:40:36.403,"Adobe Research, San Jose, CA, USA",1736,15,110,https://i.stack.imgur.com/Z99mk.jpg?s=128&g=1,http://www.francky.me
5,0,8057826,2016-08-02T15:38:27.643,baranskistad,1,5,2019-07-21T04:30:14.580,"Mankato, MN, United States",429,30,19,https://i.stack.imgur.com/Kk1s7.png?s=128&g=1,https://github.com/baranskistad
6,0,3393940,2016-08-02T15:38:27.847,Dom,30,6,2019-09-17T21:30:49.600,"Philadelphia, PA",101,12,12,https://i.stack.imgur.com/B7uhq.png?s=128&g=1,http://None
7,0,6824864,2016-08-02T15:38:28.443,Ben Clayton,0,7,2019-11-12T11:49:22.170,"Plymouth, United Kingdom",1,3,5,https://i.stack.imgur.com/4TIdl.jpg?s=128&g=1,http://benclayton.me
8,<p>I'm IT engineer programming in anything tha...,22370,2016-08-02T15:38:36.723,kenorb,3,8,2020-02-28T20:59:24.207,0,8543,581,408,0,http://ads.london
9,<p>Computer Science undergraduate at the Unive...,7263447,2016-08-02T15:38:37.287,Rob Murray,0,9,2018-10-14T10:24:20.607,United Kingdon,461,6,9,https://i.stack.imgur.com/r4U5R.jpg,https://www.linkedin.com/in/robert-murray-994a...


Users.xml dataset will be merged with Posts.xml data set 

*   They are merged from their joint parameter user id 
*   user id is "Id" parameter in User.xml data "Owner User ID" in Posts.xml data
*   Both parameters first converted into int type 
*   There were 3909 unique ids which indicated there are 3909 different users who have asked questions in this data sets 
*   So the data frames were merged according to them




In [25]:
users['Id'] = users['Id'].astype(int)

In [26]:
posts=q

In [27]:
posts['Owner User ID']=posts ['Owner User ID'].astype(int)

In [28]:
posts['Owner User ID'].nunique()

3275

In [29]:
users['Id'].nunique()

29373

In [30]:
intersection=set(users['Id']).intersection(set(posts['Owner User ID']))

In [31]:
len(intersection)

3274

New id was defined as "Owner User ID" from Posts.xml and "Id" User.xml


In [32]:
posts['New Id']=posts['Owner User ID']

In [33]:
users['New Id']=users['Id']

To eliminate confusion on naming of parameters between user and posts data, parameter names have changed such as CreationDate to User Creation Date

In [34]:
users ['User Creation Date']=users ['CreationDate']
users ['User Last Access Date']=users ['LastAccessDate']


2 data sets were merged from their joint parameter (user id)

In [35]:
new=posts.merge(users,on="New Id")

Also unnecessary Id parameters and Url parameters were eliminated.
Creation Date and LastAccessDate were dropped since their updated with other parameters name above

In [36]:
newposts=new.drop(axis=1,columns=["Owner User ID","AccountId","DisplayName","CreationDate","Id","LastAccessDate","Location","ProfileImageUrl","WebsiteUrl","AboutMe"])

Since our task is to make a prediction at the time the question asked, we did not use posts down votes and upvotes .Thus user downvotes upvotes are the only ones. For ease of use We have changed the parameter names.

In [37]:
newposts['User DownVotes']=newposts['DownVotes']
newposts['User UpVotes']=newposts['UpVotes']
newposts['User Reputation']=newposts['Reputation']
newposts['User Views']=newposts['Views']


In [38]:
newposts=newposts.drop(axis=1,columns=["DownVotes","UpVotes","Reputation","Views"])

In [39]:
newposts.head()

,CreationDate_1,CreationDate_2,CreationDate_3,Title,Body,ID,Tags,Tag Count,IsAnswered,New Id,User Creation Date,User Last Access Date,User DownVotes,User UpVotes,User Reputation,User Views
0,1.0,0.0,0.0,"What is ""backprop""?","<p>What does ""backprop"" mean? Is the ""backprop...",1,<neural-networks><backpropagation><terminology...,4,1,8,2016-08-02T15:38:36.723,2020-02-28T20:59:24.207,3,581,8543,408
1,1.0,0.0,0.0,How does noise affect generalization?,<p>Does increasing the noise in data help to i...,2,<neural-networks><machine-learning><statistica...,4,1,8,2016-08-02T15:38:36.723,2020-02-28T20:59:24.207,3,581,8543,408
2,1.0,0.0,0.0,How to find the optimal number of neurons per ...,"<p>When you're writing your algorithm, how do ...",4,<deep-network><search><neurons>,3,1,8,2016-08-02T15:38:36.723,2020-02-28T20:59:24.207,3,581,8543,408
3,1.0,0.0,0.0,What is fuzzy logic?,<p>I'm new to A.I. and I'd like to know in sim...,10,<deep-network><terminology><fuzzy-logic>,3,1,8,2016-08-02T15:38:36.723,2020-02-28T20:59:24.207,3,581,8543,408
4,1.0,0.0,0.0,"What is ""early stopping"" in machine learning?","<p>What is <a href=""https://en.wikipedia.org/w...",16,<deep-learning><definitions><overfitting><regu...,5,1,8,2016-08-02T15:38:36.723,2020-02-28T20:59:24.207,3,581,8543,408


In [40]:
newposts['User Creation Date'] = pd.to_datetime(newposts['User Creation Date'])
newposts['User Last Access Date'] = pd.to_datetime(newposts['User Last Access Date'])
newposts['New Id'] = newposts['New Id'].astype(int)
newposts['User DownVotes'] = newposts['User DownVotes'].astype(int)
newposts['User UpVotes'] = newposts['User UpVotes'].astype(int)
newposts['User Reputation'] = newposts['User Reputation'].astype(int)
newposts['User Views'] = newposts['User Views'].astype(int)
newposts['ID'] = newposts['ID'].astype(int)
newposts['IsAnswered']=newposts['IsAnswered'].astype(int)


In [41]:
newposts.dtypes


CreationDate_1                  float64
CreationDate_2                  float64
CreationDate_3                  float64
Title                            object
Body                             object
ID                                int32
Tags                             object
Tag Count                         int64
IsAnswered                        int32
New Id                            int32
User Creation Date       datetime64[ns]
User Last Access Date    datetime64[ns]
User DownVotes                    int32
User UpVotes                      int32
User Reputation                   int32
User Views                        int32
dtype: object

Label encoding on dates implemented on User creation date and user last acces date also.

In [42]:
def get_part_of_day(x):
    return (
        "1" if 5 <= int(x.strftime('%H')) <= 10
        else
        "2" if 11<=  int(x.strftime('%H'))<= 16
        else
        "3" if 17 <= int(x.strftime('%H'))  <= 22
        else
        "4"
    )
    
newposts["User Creation Date"] = newposts["User Creation Date"].apply(get_part_of_day)
newposts["User Last Access Date"] = newposts["User Last Access Date"].apply(get_part_of_day)


one hot encoding implemented on user.xml date format data

In [43]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories = "auto",sparse = False, drop = "first")
# One-Hot-Encode Class column of df
temp = ohe.fit_transform(newposts[["User Creation Date"]])


# Converting into dataframe
ohe_column = pd.DataFrame(temp, columns = ["UserCreationDate_1","UserCreationDate_2","UserCreationDate_3"])
#containating ohe column
newposts= pd.concat([newposts,ohe_column],axis = 1)


In [44]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories = "auto",sparse = False, drop = "first")
# One-Hot-Encode Class column of df
temp = ohe.fit_transform(newposts[["User Last Access Date"]])


# Converting into dataframe
ohe_column = pd.DataFrame(temp, columns = ["UserLastAccessDate_1","UserLastAccessDate_2","UserLastAccessDate_3"])
#containating ohe column
newposts= pd.concat([newposts,ohe_column],axis = 1)


In [45]:
newposts=newposts.drop(axis=1, columns=["User Creation Date","User Last Access Date"])


 xml tags are eliminated from tags to be used and preprocessing applied on text before they are added to data frame

In [46]:
newposts['Tags'] = newposts['Tags'].str.replace('<', '')
newposts['Tags'] = newposts['Tags'].str.replace('>', ' ')
newposts['Tags'] = newposts['Tags'].str.replace('-', ' ')
newposts['Tags'] = newposts['Tags'].str.split(' ')

In [47]:
newposts['Tags']

0       [neural, networks, backpropagation, terminolog...
1       [neural, networks, machine, learning, statisti...
2                      [deep, network, search, neurons, ]
3            [deep, network, terminology, fuzzy, logic, ]
4       [deep, learning, definitions, overfitting, reg...
5            [deep, network, overfitting, optimization, ]
6             [deep, network, overfitting, performance, ]
7                                      [hidden, layers, ]
8                       [deep, network, generalization, ]
9                       [deep, network, hidden, layers, ]
10                           [models, problem, solving, ]
11      [neural, networks, machine, learning, deep, ne...
12      [deep, learning, unsupervised, learning, trans...
13      [convolutional, neural, networks, image, recog...
14      [neural, networks, feedforward, neural, networ...
15      [philosophy, ethics, autonomous, vehicles, dec...
16      [deep, network, algorithm, autonomous, vehicle...
17            

In [48]:
newposts['Tags'] = newposts['Tags'].apply(lambda x: ' '.join(map(str, x)))

In [49]:
newposts=newposts[newposts['Tags'].notna()]

In [50]:
newposts = newposts.reset_index()

In [51]:
del newposts['index']

Count vectorizer method applied on tags data


In [52]:
def wm2df(wm, feat_names):
    
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
newposts.to_pickle("CQA_FRONTEND/static/data/newposts") 
cvec = CountVectorizer()
corpus = newposts['Tags'].tolist()
tags_vec = cvec.fit_transform(corpus)
tokens = cvec.get_feature_names()
wm2df(tags_vec, tokens)
newposts.insert(4,'TagsVec', tags_vec) 

In [54]:
word_df = pd.DataFrame(tags_vec.toarray(),columns=tokens)

In [55]:
tags_added=pd.concat([newposts, word_df], axis=1)

In [56]:
tags_added=tags_added.drop(axis=1,columns=["TagsVec"])

In [57]:
tags_added= tags_added.drop(axis = 1, columns = ['Tags'])

In [58]:
tags_added= tags_added.drop(axis = 1, columns = ['New Id'])

Demonstration of data frame after tags added

In [59]:
tags_added.tail(1)

,CreationDate_1,CreationDate_2,CreationDate_3,Title,Body,ID,Tag Count,IsAnswered,User DownVotes,User UpVotes,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
5760,0.0,0.0,0.0,Why gradients are so small in deep learning?,<p>The learning rate in my model is <code>0.00...,18325,1,0,0,11,...,0,0,0,0,0,0,0,0,0,0


Body(body of question) data is cleaned from xml tags

In [60]:
content = []

for data in tags_added['Body']:
    cleantext = BeautifulSoup(data, "lxml").text
    content.append(cleantext)
tags_added.insert(5,"Bodynew", content)
tags_added= tags_added.drop(axis = 1, columns = ['Body'])

Content parameter = Title of question + Body of question 

In [61]:
tags_added['Content'] = tags_added[['Title', 'Bodynew']].apply(lambda x: ' '.join(x), axis=1)

In [62]:
tags_added['Content'] = tags_added['Content'].str.lower().str.split()

In [63]:
tags_added['Content']

0       [what, is, "backprop"?, what, does, "backprop"...
1       [how, does, noise, affect, generalization?, do...
2       [how, to, find, the, optimal, number, of, neur...
3       [what, is, fuzzy, logic?, i'm, new, to, a.i., ...
4       [what, is, "early, stopping", in, machine, lea...
5       [what, are, the, methods, of, optimizing, over...
6       [what, is, the, "dropout", technique?, what, p...
7       [what, is, the, purpose, of, the, hidden, laye...
8       [how, can, generalization, error, be, estimate...
9       [what, kind, of, problems, require, more, than...
10      [what, are, the, real, world, uses, for, sat, ...
11      [how, is, a, deep, neural, network, different,...
12      [why, does, unsupervised, pre-training, help, ...
13      [how, is, it, possible, that, deep, neural, ne...
14      [what, is, the, significance, of, weights, in,...
15      [how, could, self-driving, cars, make, ethical...
16      [which, machine, learning, algorithm, is, used...
17      [what,

Doc2vec method implemented on Content data

In [64]:
# Set file names for train and test data

def read_corpus(fname, tokens_only=False):
        for i, line in enumerate(fname):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])



In [65]:
tags_added = tags_added[tags_added['Content'].notna()]

In [66]:
tags_added_fix = tags_added.reset_index()

In [67]:
del tags_added_fix['index']

In [68]:
from sklearn.model_selection import train_test_split
vecList = []
for index, row in tags_added_fix.iterrows():
  train_corpus = list(read_corpus(row['Content']))
 
  model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
  model.build_vocab(train_corpus)
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  vector = model.infer_vector(row['Content'])
  #print(vector)
  vecList.append(vector)

Vector size is defined as 100. Each vector has been added to the data frame for corresponding questions

In [69]:
col_list_content = ['content' + str(x) for x in range(0,100)]


In [70]:
doc2vecdf = pd.DataFrame(vecList,columns=col_list_content)


In [71]:
final_train=pd.concat([tags_added_fix, doc2vecdf], axis=1)

In [72]:
final_train.sort_index(axis=1, inplace=True)

To make a better prediction data is shuffled 


In [73]:
shuffled_final=final_train.sample(frac=1)


In [74]:
shuffled_final = shuffled_final.reset_index()

In [75]:
shuffled_final=shuffled_final.drop(axis=1, columns=["index","ID","Content","Bodynew","Title"])

In [76]:
shuffled_final=shuffled_final.fillna(0.0) 

In [77]:
shuffled_final.sort_index(axis=1, inplace=True)

In [78]:
shuffled_final.replace(-9223372036854775808,0)

,CreationDate_1,CreationDate_2,CreationDate_3,IsAnswered,Tag Count,User DownVotes,User Reputation,User UpVotes,User Views,UserCreationDate_1,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
0,0.0,0.0,0.0,1,2,0,43,1,7,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,0.0,1,1,29,5607,3179,827,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.0,0.0,0,1,0,21,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,0.0,1,4,0,41,0,1,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,1.0,1,3,0,159,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0
5,0.0,0.0,0.0,1,4,0,99,0,3,0.0,...,0,0,0,0,0,0,0,0,0,0
6,0.0,1.0,0.0,1,4,0,111,2,0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,0.0,0.0,0.0,1,2,0,247,0,3,0.0,...,0,0,0,0,0,0,0,0,0,0
8,0.0,0.0,1.0,1,1,10,787,204,8,0.0,...,0,0,0,0,0,0,0,0,0,0
9,0.0,1.0,0.0,1,1,0,43,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0


  This is where we define the test set, for the user interface it will be dynamic parameter and updated for each question will be asked , however at this stage we have define the question manually.

In [79]:
derived_question_body ="As far I know, the RNN accepts a sequence as input and can produce as a sequence as output.Are there neural networks that accept graphs or trees as inputs, so that to represent the relationships between the nodes of the graph or tree?"
derived_title="Are there neural networks that accept graphs or trees as inputs?"
derived_creation_date="2016-08-02T15:41:22.020"
derived_tags="<neural-networks><graphs>"
derived_userReputation="344"
derived_userCreationDate="2016-08-02T15:38:36.723"
derived_userLastAccessDate="2019-11-30T19:21:45.687"
derived_userViews="30" 
derived_userUpVotes="0" 
derived_userDownVotes="0"
# create a new data frame 

df = pd.DataFrame({'Question': [derived_question_body],'Title' :[derived_title],'Creation Date':[derived_creation_date],'Tags':[derived_tags],'User Reputation': [derived_userReputation],"User Creation Date" : [derived_userCreationDate],"User Last Access Date" : [derived_userLastAccessDate],"User Views": [derived_userViews],"User UpVotes": [derived_userUpVotes],"User DownVotes": [derived_userDownVotes]}) 
   
df

,Question,Title,Creation Date,Tags,User Reputation,User Creation Date,User Last Access Date,User Views,User UpVotes,User DownVotes
0,"As far I know, the RNN accepts a sequence as i...",Are there neural networks that accept graphs o...,2016-08-02T15:41:22.020,<neural-networks><graphs>,344,2016-08-02T15:38:36.723,2019-11-30T19:21:45.687,30,0,0


Same preprocessing applied on Date format data 

*   Label encoding
*   One hot encoding

Since there is only 1 question on test set that means there will be only 1 part of the day to encode . So one-hot-encoding would not be able to create columns for each part of the day . We have written a function to convert label encoded parameter to one hot encoded parameters.



In [80]:
df['Creation Date'] = pd.to_datetime(df['Creation Date'])
df['User Creation Date'] = pd.to_datetime(df['User Creation Date'])
df['User Last Access Date'] = pd.to_datetime(df['User Last Access Date'])
df['User DownVotes'] = df['User DownVotes'].astype(int)
df['User UpVotes'] = df['User UpVotes'].astype(int)
df['User Reputation'] = df['User Reputation'].astype(int)
df['User Views'] = df['User Views'].astype(int)



In [81]:
df= df.fillna(0)

Label encoding

In [82]:
def get_part_of_day(x):
    return (
        "1" if 5 <= int(x.strftime('%H')) <= 10
        else
        "2" if 11<=  int(x.strftime('%H'))<= 16
        else
        "3" if 17 <= int(x.strftime('%H'))  <= 22
        else
        "4"
    )
    
df["Creation Date"] = df["Creation Date"].apply(get_part_of_day)
df["User Creation Date"] = df["User Creation Date"].apply(get_part_of_day)
df["User Last Access Date"] = df["User Last Access Date"].apply(get_part_of_day)


One hot encoding function for the date format data

In [83]:
def encoder(x):
  a = 0.0
  b = 0.0
  c = 0.0

  if(x == "1"):
    return a,b,c
  elif (x == "2"):
    a = 1.0
    return a,b,c
  elif (x == "3"):
    b = 1.0
    return a,b,c
  else:
    c = 1.0
    return a,b,c

df["CreationDate_1"], df["CreationDate_2"], df["CreationDate_3"] = zip(*df['Creation Date'].map(encoder))
df["UserCreationDate_1"], df["UserCreationDate_2"], df["UserCreationDate_3"] = zip(*df['User Creation Date'].map(encoder))
df["UserLastAccessDate_1"], df["UserLastAccessDate_2"], df["UserLastAccessDate_3"] = zip(*df['User Last Access Date'].map(encoder))

Tag count calculated 

In [84]:
df['Tag Count'] = df['Tags'].str.count('<')

Tags data is cleaned from xml format

In [85]:
df['Tags'] = df['Tags'].str.replace('<', '')
df['Tags'] = df['Tags'].str.replace('>', ' ')
df['Tags'] = df['Tags'].str.replace('-', ' ')
df['Tags'] = df['Tags'].str.split(' ')

To create same data frame for the test data , we have used the tokens that we created from train set's tags data. Updated dataframe according to the existence of tag

In [86]:
alltags = pd.DataFrame(0, index=np.arange(1), columns=tokens)
for tag in df.iloc[0]['Tags']:
  alltags[tag] = 1



In [87]:
alltags
q_tags_added=pd.concat([df, alltags], axis=1)

In [88]:
q_tags_added = q_tags_added.iloc[:, :-1]

Data frame after tags are added 

In [89]:
q_tags_added

,Question,Title,Creation Date,Tags,User Reputation,User Creation Date,User Last Access Date,User Views,User UpVotes,User DownVotes,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
0,"As far I know, the RNN accepts a sequence as i...",Are there neural networks that accept graphs o...,2,"[neural, networks, graphs, ]",344,2,3,30,0,0,...,0,0,0,0,0,0,0,0,0,0


Doc2vec method implemented in same manner as train set
body and title of the question used together to create vectoral representation of question.

In [90]:
#words kolonu title ile bodynin birleşmiş hali, 
q_tags_added['Content'] = q_tags_added[['Title', 'Question']].apply(lambda x: ' '.join(x), axis=1)

In [91]:
q_tags_added['Content'] = q_tags_added['Content'].str.lower().str.split()

In [92]:
# Set file names for train and test data

def read_corpus(fname, tokens_only=False):
        for i, line in enumerate(fname):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
                import numpy as np

In [93]:
from sklearn.model_selection import train_test_split
# choose question posts and take body and convert list and with the help of corpus function each question will be presented as documents.

#test_corpus = list(read_corpus(b[b["PostTypeId"]==1]["Body"], tokens_only=True))
q_vecList = []
for index, row in q_tags_added.iterrows():

  train_corpus = list(read_corpus(row['Content']))
  model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
  model.build_vocab(train_corpus)
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  vector = model.infer_vector(row['Content'])
  #print(vector)
  q_vecList.append(vector)

In [94]:
q_col_list = ['content' + str(x) for x in range(0,100)]

In [95]:
q_doc2vecdf = pd.DataFrame(q_vecList,columns=q_col_list)

In [96]:
final_test=pd.concat([q_tags_added, q_doc2vecdf], axis=1)

In [97]:
final_test=final_test.drop(axis=1,columns=["Content","Question","Title","Creation Date","Tags","User Creation Date","User Last Access Date"])

In [98]:
final_test=final_test.reset_index()

In [99]:
del final_test['index']

In [100]:
final_test=final_test.replace(-9223372036854775808,0)

In [101]:
final_test.sort_index(axis=1, inplace=True)

For each question their corresponding 100 columns vector representation added to dataframe . This is final version of test data

In [102]:
final_test

,CreationDate_1,CreationDate_2,CreationDate_3,Tag Count,User DownVotes,User Reputation,User UpVotes,User Views,UserCreationDate_1,UserCreationDate_2,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
0,1.0,0.0,0.0,2,0,344,0,30,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
shuffled_final.head(1)

,CreationDate_1,CreationDate_2,CreationDate_3,IsAnswered,Tag Count,User DownVotes,User Reputation,User UpVotes,User Views,UserCreationDate_1,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
0,0.0,0.0,0.0,1,2,0,43,1,7,0.0,...,0,0,0,0,0,0,0,0,0,0


There are total 5046 questions when we merged user.xml data and posts.xml data . 3811 of them is answered ,1235 of them is not answered

In [104]:
shuffled_final.groupby("IsAnswered").count()

,CreationDate_1,CreationDate_2,CreationDate_3,Tag Count,User DownVotes,User Reputation,User UpVotes,User Views,UserCreationDate_1,UserCreationDate_2,...,weight,weights,wetware,winter,word,word2vec,words,world,yolo,zero
IsAnswered,,,,,,,,,,,,,,,,,,,,,
0,1565,1565,1565,1565,1565,1565,1565,1565,1565,1565,...,1565,1565,1565,1565,1565,1565,1565,1565,1565,1565
1,4196,4196,4196,4196,4196,4196,4196,4196,4196,4196,...,4196,4196,4196,4196,4196,4196,4196,4196,4196,4196


In [105]:
shuffled_final.to_pickle("CQA_FRONTEND/static/data/shuffled_perc")  # where to save it, usually as a .pkl

In [106]:
xtrain = shuffled_final.drop(axis = 1, columns=['IsAnswered'])
labels = shuffled_final['IsAnswered']

For the question that has been used as test question: The model predicted that, the question will be answered with 83% probability. This is the final model result for the user interface implementation. Different models have been tried and results of them has shown in the report. This model worked with ≈ 80% accuracy when the data is divided into train and test set.

In [107]:
# fit final model
rfm=RandomForestClassifier(bootstrap = True,n_estimators=100,min_samples_leaf=1,
                           random_state=50, max_depth= 50,min_samples_split= 20)

rfm.fit(xtrain,labels)

ynew = rfm.predict_proba(final_test)
# show the inputs and predicted outputs
for i in range(len(final_test)):
	print(" Predicted=%s" % (ynew[i]))

 Predicted=[0.25950955 0.74049045]


The printed results show that

* The question is predicted to be answered with ≈ 81% probability
* The question is predicted to be not answered with ≈ 19% probability